In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from matplotlib.patches import Circle
import os
import pickle
from scipy.stats import mannwhitneyu, ttest_ind, kruskal, f_oneway
from statannotations.Annotator import Annotator
from itertools import combinations
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import clear_output, display
import sys
sys.path.append('../scripts') 
from functions import *
from config import nutrient_info, conversion_factors

In [2]:
df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')

C:\Users\Sophie\AppData\Local\Temp\ipykernel_18096\2711278687.py:1: DtypeWarning: Columns (1,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_food_scaled = pd.read_csv('../data/df_food_scaled.csv')


In [3]:
qualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'qualifying'}
disqualifying_nutrients = {nutr: info for nutr, info in nutrient_info.items() if info['type'] == 'disqualifying'}

In [4]:
df_salty_snacks = df_food_scaled[df_food_scaled['food_group'] == 'salty_snacks']
df_salty_snacks.drop_duplicates('combined_name')[['combined_name', 'QI', 'DI', 'NB']].sort_values('QI', ascending=False)

,combined_name,QI,DI,NB
303368,Roasted hot green peas,2.982920,0.211518,79.166667
279357,Cashew-Peanut-Mix : mit Honig und Salz,1.158471,0.680666,63.866804
284650,"coop FINE FOOD Erdnüsse geröstet, mit Wasabi",1.141059,0.551386,62.895906
280659,Party - Mexico Nuts,1.125503,0.643868,64.045639
283693,Coop Fine Food Wasabi Coated Peanuts,1.124642,0.629247,62.373518
...,...,...,...,...
314871,Chips de Crevettes,0.015816,0.570205,1.581624
284379,BITE SIZE TWISTS,0.015528,0.304782,1.552795
276105,Pom-Bär Original,0.014746,0.523004,1.474553
312493,Pombär - Original,0.013281,0.326287,1.328125


In [5]:
high_qi_salty_snacks = df_salty_snacks[df_salty_snacks['QI'] > 1].drop_duplicates('combined_name').sort_values('QI', ascending=False)

In [6]:
high_qi_salty_snacks[['combined_name', 'eaten_quantity', 'QI', 'DI', 'NB']]

,combined_name,eaten_quantity,QI,DI,NB
303368,Roasted hot green peas,50.0,2.982920,0.211518,79.166667
279357,Cashew-Peanut-Mix : mit Honig und Salz,40.0,1.158471,0.680666,63.866804
284650,"coop FINE FOOD Erdnüsse geröstet, mit Wasabi",40.0,1.141059,0.551386,62.895906
280659,Party - Mexico Nuts,100.0,1.125503,0.643868,64.045639
283693,Coop Fine Food Wasabi Coated Peanuts,45.0,1.124642,0.629247,62.373518
302702,Bio Popcorn,50.0,1.095092,0.086464,62.667669
307341,"Lorenz - Nic Nac's, the double-crunch-peanuts",31.0,1.070549,0.771778,61.658511
284741,Terroir urbain - Croustilles De Sion,30.0,1.025128,0.933852,68.553028
254444,Popcorn salted,30.0,1.001510,0.083911,59.877873


In [7]:
ratio_cols = [i for i in high_qi_salty_snacks.columns if '_ratio_scaled' in i]

df_corr = high_qi_salty_snacks[ratio_cols + ['QI']]
corr = df_corr.corr()['QI']
corr.sort_values(ascending=True)

vitamin_e_activity_eaten_ratio_scaled                       -0.524559
fatty_acids_saturated_eaten_ratio_scaled                    -0.462559
fat_eaten_ratio_scaled                                      -0.347533
salt_eaten_ratio_scaled                                     -0.338632
vitamin_b12_eaten_ratio_scaled                              -0.171696
cholesterol_eaten_ratio_scaled                              -0.164671
vitamin_d_eaten_ratio_scaled                                -0.164671
all_trans_retinol_equivalents_activity_eaten_ratio_scaled   -0.164671
sodium_eaten_ratio_scaled                                   -0.162907
magnesium_eaten_ratio_scaled                                -0.030563
niacin_eaten_ratio_scaled                                    0.167542
zinc_eaten_ratio_scaled                                      0.339231
calcium_eaten_ratio_scaled                                   0.339232
pantothenic_acid_eaten_ratio_scaled                          0.370493
sugar_eaten_ratio_sc

In [8]:
def interactive_exclude_nutrient(nutrient):
    clear_output(wait=True)
    df_plot = compare_qi_excluding_nutrient(
        df=high_qi_salty_snacks, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )

    display(df_plot[['combined_name', 'QI', f"QI_excl_{nutrient}", 'QI_diff', 'QI_pct_change']])

    print(f"\nExcluding {nutrient} for NB:")
    df_nb = compare_nb_excluding_nutrient(
        df=high_qi_salty_snacks, 
        nutrient_to_exclude=nutrient,
        qualifying_nutrients=qualifying_nutrients, 
        scaling_factor=2000
    )
    display(df_nb[['combined_name', 'NB', f"NB_excl_{nutrient}", 'NB_diff', 'NB_pct_change']])

nutrient_choice = ['folate_eaten', 
                   'niacin_eaten', 
                   'pantothenic_acid_eaten', 
                   'vitamin_b2_eaten', 
                   'vitamin_b1_eaten', 
                   'all_trans_retinol_equivalents_activity_eaten', 
                   'beta_carotene_eaten', 
                   'vitamin_b12_eaten', 
                   'vitamin_b6_eaten', 
                   'vitamin_c_eaten', 
                   'vitamin_d_eaten', 
                   'vitamin_e_activity_eaten', 
                   'calcium_eaten', 
                   'iron_eaten', 
                   'magnesium_eaten', 
                   'phosphorus_eaten', 
                   'potassium_eaten', 
                   'zinc_eaten', 
                   'fiber_eaten', 
                   'protein_eaten']

In [9]:
interact(interactive_exclude_nutrient, nutrient=widgets.Dropdown(options=nutrient_choice, description='Exclude:'))

interactive(children=(Dropdown(description='Exclude:', options=('folate_eaten', 'niacin_eaten', 'pantothenic_a…

<function __main__.interactive_exclude_nutrient(nutrient)>

- niacin
- pantothenic acid
- vitamin b1
- beta carotene
- vitamin e
- magnesium
- phosphorus
- zinc
- fiber
- protein